In [2]:
import tensorflow as tf
import numpy as np
import math


def get_batch(x, y, size):
    data = tf.concat([x, y], 1)
    data = tf.random.shuffle(data)
    batch_x = tf.slice(data, begin=[0, 0], size=[size, tf.shape(x)[1]])
    batch_y = tf.slice(data, begin=[0, tf.shape(x)[1]], size=[size, tf.shape(y)[1]])
    return batch_x, batch_y


class GRF_2d():
    def __init__(self, N, sigma=2., tau=2.):
        self.nr_spacediscr = N
        dx = 1. / N
        self.dx = dx
        self.sigma = sigma
        self.tau = tau
        help_tensor = np.zeros((N, N))
        help_tensor[0, 0] = - 4. / (dx ** 2)
        help_tensor[1, 0] = 1. / dx ** 2
        help_tensor[0, 1] = 1. / dx ** 2
        help_tensor[0, -1] = 1. / dx ** 2
        help_tensor[-1, 0] = 1. / dx ** 2
        help_tf_tensor = tf.convert_to_tensor(help_tensor, dtype=tf.float64)
        lap_tensor_help = tf.stack([tf.roll(help_tf_tensor, i, axis=1) for i in range(N)])
        lap_tensor = tf.stack([tf.roll(lap_tensor_help, i, axis=1) for i in range(N)])
        lap_matrix = tf.reshape(lap_tensor, [N * N, N * N])
        help_matrix = - lap_matrix + tau * tf.eye(N * N, dtype=tf.float64)
        self.lu, self.perm = tf.linalg.lu(help_matrix, output_idx_type=tf.int64)

    def sample(self, batch_size):
        coeff = self.sigma * self.nr_spacediscr * tf.random.normal([self.nr_spacediscr ** 2, batch_size], dtype=tf.float64)
        field = tf.transpose(tf.linalg.lu_solve(self.lu, self.perm, coeff))
        return field

In [3]:
from tensorflow import keras


################################################################
# This is the Fourier neural operator for 2d input (based on pytorch implementation of Zongyi Li)
################################################################
class SpectralConv2d(tf.keras.Model):

    def __init__(self, in_dim, out_dim, fourier_modes_1, fourier_modes_2, **kwargs):
        super().__init__(**kwargs)

        self.in_nodes = in_dim
        self.out_dim = out_dim
        self.fm_1 = fourier_modes_1  # Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.fm_2 = fourier_modes_2

        self.scale = (1 / (in_dim * out_dim))
        init_weights_1 = tf.cast(self.scale * tf.random.normal([in_dim, out_dim, fourier_modes_1, fourier_modes_2]),
                               dtype=tf.complex128)
        init_weights_2 = tf.cast(self.scale * tf.random.normal([in_dim, out_dim, fourier_modes_1, fourier_modes_2]),
                               dtype=tf.complex128)
        self.FNO_weights_1 = tf.Variable(init_weights_1, dtype=tf.complex128)
        self.FNO_weights_2 = tf.Variable(init_weights_2, dtype=tf.complex128)

    # Complex multiplication
    def compl_mul2d(self, inputs, weights):
        # (batch, in_channel, x, y ), (in_channel, out_channel, x, y) -> (batch, out_channel, x, y)
        return tf.einsum("bikl,ijkl->bjkl", inputs, weights)

    def call(self, inputs):
        inputs = tf.cast(inputs, dtype=tf.float64)
        batchsize = tf.shape(inputs)[0]
        # Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = tf.signal.rfft2d(inputs)
        # Multiply relevant Fourier modes
        out_ft = tf.zeros([batchsize, self.out_dim, tf.shape(inputs)[-2], tf.shape(inputs)[-1] // 2 + 1], dtype=tf.complex128)
        out_ft_1 = self.compl_mul2d(x_ft[:, :, :self.fm_1, :self.fm_2], self.FNO_weights_1)
        out_ft_2 = self.compl_mul2d(x_ft[:, :, -self.fm_1:, :self.fm_2], self.FNO_weights_2)
        out_ft = tf.concat([out_ft_1, tf.zeros([batchsize, self.out_dim, tf.shape(inputs)[-2] - 2 * self.fm_1, self.fm_2],
                                               dtype=tf.complex128), out_ft_2], axis=-2)
        out_ft = tf.concat([out_ft, tf.zeros([batchsize, self.out_dim, tf.shape(inputs)[-2],
                                              tf.shape(inputs)[-1] // 2 + 1 - self.fm_2], dtype=tf.complex128)], axis=-1)
        # Return to physical space
        x = tf.signal.irfft2d(out_ft, fft_length=[tf.shape(inputs)[-2], tf.shape(inputs)[-1]])
        return x


class FNO2d_model(tf.keras.Model):

    def __init__(self, fourier_modes_1, fourier_modes_2, width, depth, **kwargs):
        super().__init__(**kwargs)

        self.fm_1 = fourier_modes_1
        self.fm_2 = fourier_modes_2
        self.width = width
        self.depth = depth

        self.fc0 = keras.layers.Dense(self.width,
                                      dtype=tf.float64)
        # Size of input will be determined later. input channel is 2: (a(x), x)

        self.convs = []
        self.ws = []

        for l in range(self.depth):
            conv = SpectralConv2d(self.width, self.width, self.fm_1, self.fm_2)
            w = keras.layers.Conv2D(filters=self.width, kernel_size=1, dtype=tf.float64)

            self.convs = self.convs + [conv]
            self.ws = self.ws + [w]

        self.fc1 = keras.layers.Dense(128, dtype=tf.float64)
        self.fc2 = keras.layers.Dense(1, dtype=tf.float64)

    def call(self, inputs):
        inputs = tf.cast(inputs, dtype=tf.float64)

        grid = self.get_grid(tf.shape(inputs))
        x = tf.cast(inputs, tf.float64)
        x = tf.concat([x, grid], axis=-1)
        x = self.fc0(x)

        x = tf.transpose(x, perm=[0, 3, 1, 2])
        for l in range(self.depth - 1):
            x1 = self.convs[l](x)
            x2 = tf.transpose(self.ws[l](tf.transpose(x, perm=[0, 2, 3, 1])), perm=[0, 3, 1, 2])
            x = x1 + x2
            x = tf.nn.gelu(x)

        x1 = self.convs[-1](x)
        x2 = tf.transpose(self.ws[-1](tf.transpose(x, perm=[0, 2, 3, 1])), perm=[0, 3, 1, 2])
        x = x1 + x2
        x = tf.transpose(x, perm=[0, 2, 3, 1])

        x = self.fc1(x)
        x = tf.nn.gelu(x)
        x = self.fc2(x)
        return x

    def get_grid(self, shape):
        batch_size, size_x, size_y = shape[0], shape[1], shape[2]  # Size_inputs corresponds to the space discretization
        gridx = tf.constant(np.linspace(0, 1, size_x), dtype=tf.float64)
        gridx = tf.reshape(gridx, (1, size_x, 1, 1))
        gridx = tf.tile(gridx, [batch_size, 1, size_y, 1])
        gridy = tf.constant(np.linspace(0, 1, size_y), dtype=tf.float64)
        gridy = tf.reshape(gridy, (1, 1, size_x, 1))
        gridy = tf.tile(gridy, [batch_size, size_x, 1, 1])
        return tf.concat([gridx, gridy], axis=-1)

In [2]:
import time


class semilinear_PDE_2d():
    #  Class implementing semilinear PDE on a two-dimensional square grid.
    def __init__(self, linear_operator, coarse_linear_op, nonlin, nonlin_name):
        self.linear_operator = linear_operator
        self.coarse_linear_op = coarse_linear_op
        self.nonlin = nonlin
        self.nonlin_name = nonlin_name
        self.space_dim, _ = linear_operator.shape
        self.nr_spacediscr = int(tf.sqrt(tf.cast(self.space_dim, tf.float64)))
        self.dx = 1. / self.nr_spacediscr
        self.coarse_space_dim, _ = coarse_linear_op.shape  # consider same PDE on coarser grid (for faster ADANN training)
        self.coarse_spacediscr = int(tf.sqrt(tf.cast(self.coarse_space_dim, tf.float64)))
        self.coarse_dx = 1. / self.coarse_spacediscr

        self.ODE_name = f'Semilinear PDE on {self.nr_spacediscr}x{self.nr_spacediscr}-grid with nonlinearity {self.nonlin_name}'


def semilinear_heat_PDE_FDM(nr_spacediscr, coarse_spacediscr, laplace_factor, nonlin, nonlin_name):
    # creates a semilinear PDE object with linear term of the form nu * Laplace on domain [0,1]^2.
    if coarse_spacediscr is None:
        coarse_spacediscr = nr_spacediscr
    if nr_spacediscr % coarse_spacediscr != 0:
        raise ValueError(f'Sorry, approximate resolution {coarse_spacediscr} must divide original resolution {nr_spacediscr}.')
    dx = 1. / nr_spacediscr
    operator = laplace_factor * laplace_matrix(nr_spacediscr, dx)  # laplace-matrix on original grid
    coarse_dx = 1. / coarse_spacediscr
    coarse_operator = laplace_factor * laplace_matrix(coarse_spacediscr, coarse_dx)  # laplace-matrix on coarser grid
    semilin_heat_pde = semilinear_PDE_2d(operator, coarse_operator, nonlin, nonlin_name)
    semilin_heat_pde.ODE_name = f'Semilinear heat PDE on {nr_spacediscr}x{nr_spacediscr}-grid, Lap-factor: {laplace_factor}, nonlin: {nonlin_name}'

    return semilin_heat_pde


def allen_cahn_nonlin(inputs):
    return inputs - inputs * inputs * inputs


def sin_nonlin(inputs):
    return tf.math.sin(inputs)


def zero_nonlin(inputs):
    return tf.zeros_like(inputs)


def AJ_nonlin(inputs):
    input_norm_squared = tf.square(inputs)
    factor = tf.math.reciprocal(1 + input_norm_squared)
    return (1 - inputs) * factor

def sqrt_nonlin(inputs):
    input_norm_squared = tf.square(inputs)
    return tf.sqrt(1. + input_norm_squared)


def volterra_nonlin(inputs):
    return inputs - inputs * inputs


def exponential_nonlin(inputs):
    input_norm_squared = tf.square(inputs)
    return 1. - tf.exp(- input_norm_squared)


def laplace_matrix(N, dx):
    # assembles matrix representing the discrete Laplace operator on NxN-grid with periodic boundary conditions
    help_tensor = np.zeros((N, N))
    help_tensor[0, 0] = - 4. / (dx ** 2)
    help_tensor[1, 0] = 1. / dx ** 2
    help_tensor[0, 1] = 1. / dx ** 2
    help_tensor[0, -1] = 1. / dx ** 2
    help_tensor[-1, 0] = 1. / dx ** 2
    help_tf_tensor = tf.convert_to_tensor(help_tensor, dtype=tf.float64)
    lap_tensor_help = tf.stack([tf.roll(help_tf_tensor, i, axis=1) for i in range(N)])
    lap_tensor = tf.stack([tf.roll(lap_tensor_help, i, axis=1) for i in range(N)])
    lap_matrix = tf.reshape(lap_tensor, [N * N, N * N])
    return lap_matrix


def pde_step_rk2(pde, w, dt, inverse, rk_params, sparse):
    alpha, beta = rk_params
    F = pde.nonlin(w)
    first = tf.matmul(w, pde.linear_operator, b_is_sparse=sparse) + F
    k1 = tf.matmul(first, inverse)
    help1 = tf.matmul(w + 2. * dt * alpha * (0.5 - beta) * k1, pde.linear_operator, b_is_sparse=sparse)
    w2 = w + dt * alpha * k1
    help2 = pde.nonlin(w2)
    k2 = tf.matmul(help1 + help2, inverse)
    w_new = w + dt * ((1. - 0.5 / alpha) * k1 + 0.5 / alpha * k2)
    return w_new


def solve_2d_pde_rk2(pde, w0, T_end, nt, rk_params, verbose=False, sparse=False): 
# solves PDE with linear implicit 2nd order Runge-Kutta scheme
# two positive real parameters alpha, beta
    dt = T_end / nt   
    inverse = tf.linalg.inv(tf.eye(pde.space_dim, dtype=tf.float64) - rk_params[1] * dt * pde.linear_operator)
    for i in range(nt):
        w = pde_step_rk2(pde, w0, dt, inverse, rk_params, sparse)
        w0 = w
        if verbose:
            if (i + 1) % 10 == 0:
                print(f'Number of time-steps: {i + 1}')
                print(f'L-infty norm of solution: {tf.reduce_max(tf.abs(w0))}')
    return w0

In [ ]:
from keras.utils.layer_utils import count_params, print_summary


def project_matrix(N, k):
    help_tensor = np.zeros((k * N, k * N))
    help_tensor[0, 0] = 1.
    help_tf_tensor = tf.convert_to_tensor(help_tensor, dtype=tf.float64)
    refine_tensor_help = tf.stack([tf.roll(help_tf_tensor, k * i, axis=1) for i in range(N)])
    refine_tensor = tf.stack([tf.roll(refine_tensor_help, k * i, axis=1) for i in range(N)])
    return tf.transpose(tf.reshape(refine_tensor, [N ** 2, k ** 2 * N ** 2]))


def generate_reference_solution(pde_model, input):
    algo = pde_model.ref_algo
    if algo == 'RK2':
        output = solve_2d_pde_rk2(pde_model.pde, input, pde_model.T_end, pde_model.reference_timesteps, pde_model.reference_params)
    else:
        raise ValueError(f'Sorry, the reference algorithm {algo} was not defined.')

    return output


class PDE_RK2_model(tf.keras.Model):  # model implementing ADANNs for the above second order Runge-Kutta scheme
    def __init__(self, pde, T_end, nr_timesteps, reference_timesteps, params, ref_algo, reference_params=None,
                 train_refine=True, learning_rate=0.001, **kwargs):
        super().__init__(**kwargs)

        self.T_end = T_end
        self.pde = pde
        self.nr_spacediscr = pde.nr_spacediscr
        self.spacedim = pde.space_dim
        self.coarse_spacediscr = pde.coarse_spacediscr
        self.coarse_spacedim = pde.coarse_space_dim
        self.refine_factor = self.nr_spacediscr // self.coarse_spacediscr

        self.linear_operator = pde.linear_operator
        self.coarse_linear_op = pde.coarse_linear_op
        self.nonlin = pde.nonlin

        self.nr_timesteps = nr_timesteps
        self.reference_timesteps = reference_timesteps
        self.reference_params = reference_params
        self.ref_algo = ref_algo
        self.alpha, self.beta = params
        self.learning_rate = learning_rate

        self.dt = self.T_end / self.nr_timesteps

        if self.coarse_spacediscr == self.nr_spacediscr:
            self.modeltype = f'RK2_Model with depth {nr_timesteps} and init params {params}'
        else:
            self.modeltype = f'RK2_Model with depth {nr_timesteps} and init params {params} trained on coarser {self.coarse_spacediscr}x{self.coarse_spacediscr}-grid'

        inverse = tf.linalg.inv(tf.eye(self.coarse_spacedim, dtype=tf.float64) - self.beta * self.dt * self.coarse_linear_op)
        linear_flow_1_init = tf.matmul(tf.eye(self.coarse_spacedim, dtype=tf.float64) + self.dt * (1. - self.beta) * self.coarse_linear_op, inverse) + \
            self.dt * self.dt * (0.5 - self.beta) * tf.matmul(
            tf.matmul(self.coarse_linear_op, inverse), tf.matmul(self.coarse_linear_op, inverse))
        nonlinear_flow_1_init = self.dt * (1. - 0.5 / self.alpha) * inverse + \
            self.dt * self.dt * (0.5 - self.beta) * tf.matmul(inverse, tf.matmul(self.coarse_linear_op, inverse))
        nonlinear_flow_2_init = self.dt / (2. * self.alpha) * inverse
        linear_flow_2_init = tf.matmul(
            tf.eye(self.coarse_spacedim, dtype=tf.float64) + self.dt * (self.alpha - self.beta) * self.coarse_linear_op, inverse)
        nonlinear_flow_3_init = self.dt * self.alpha * inverse
        # create trainable network weight matrices
        self.linear_flow_1_weights = [tf.Variable(linear_flow_1_init, dtype=tf.float64) for _ in
                                      range(self.nr_timesteps)]
        self.nonlinear_flow_1_weights = [tf.Variable(nonlinear_flow_1_init, dtype=tf.float64) for _ in
                                         range(self.nr_timesteps)]
        self.nonlinear_flow_2_weights = [tf.Variable(nonlinear_flow_2_init, dtype=tf.float64) for _ in
                                         range(self.nr_timesteps)]
        self.linear_flow_2_weights = [tf.Variable(linear_flow_2_init, dtype=tf.float64) for _ in
                                      range(self.nr_timesteps)]
        self.nonlinear_flow_3_weights = [tf.Variable(nonlinear_flow_3_init, dtype=tf.float64) for _ in
                                         range(self.nr_timesteps)]

        self.project_matrix = project_matrix(self.coarse_spacediscr, self.refine_factor)

        # Training
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        self.evaluations = []

        self.initializer = GRF_2d(self.nr_spacediscr)

    def call(self, initial_values):

        u = tf.cast(initial_values, tf.float64)
        u = tf.matmul(u, self.project_matrix)
        for m in range(self.nr_timesteps):
            inner_stage = tf.matmul(u, self.linear_flow_2_weights[m]) \
                          + tf.matmul(self.nonlin(u), self.nonlinear_flow_3_weights[m])
            u = tf.matmul(u, self.linear_flow_1_weights[m]) + tf.matmul(self.nonlin(u), self.nonlinear_flow_1_weights[m]) \
                + tf.matmul(self.nonlin(inner_stage), self.nonlinear_flow_2_weights[m])

        return u



class FNO_Model(tf.keras.Model): 
    '''
      Model implementing FNO for two-dimensional semilinear ODEs based on the class in FNO_2d
    '''

    def __init__(self, pde, T_end, FNO_dims, reference_timesteps, ref_algo, reference_params=None, learning_rate=0.001, **kwargs):
        super().__init__(**kwargs)

        self.modeltype = f'FNO_Model with architecture {FNO_dims}'

        self.T_end = float(T_end)
        self.nr_fourier_modes, self.width, self.depth = FNO_dims
        self.learning_rate = learning_rate

        self.pde = pde
        self.nr_spacediscr = pde.nr_spacediscr
        self.spacedim = pde.space_dim

        self.coarse_spacediscr = pde.coarse_spacediscr
        self.coarse_spacedim = pde.coarse_space_dim
        self.refine_factor = self.nr_spacediscr // self.coarse_spacediscr

        self.project_matrix = project_matrix(self.coarse_spacediscr, self.refine_factor)

        self.reference_error = tf.Variable(1., dtype=tf.float64, trainable=False)

        self.reference_timesteps = reference_timesteps
        self.reference_params = reference_params
        self.ref_algo = ref_algo

        # Create network model
        self.fno_model = FNO2d_model(self.nr_fourier_modes, self.nr_fourier_modes, self.width, self.depth)

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

        self.evaluations = []

        self.initializer = GRF_2d(self.nr_spacediscr)

    def call(self, initial_values):
        '''
            initial_values: [batch_size, space_dim]
        '''
        u = tf.cast(initial_values, tf.float64)
        u = tf.matmul(u, self.project_matrix)
        u = tf.reshape(u, [-1, self.coarse_spacediscr, self.coarse_spacediscr])
        u = tf.expand_dims(u, -1)
        u = self.fno_model.call(u)
        u = tf.squeeze(u, -1)
        u = tf.reshape(u, [-1, self.coarse_spacedim])
        return u


class ANN_Model(tf.keras.Model):
# standard fully connected feedforward neural network
    def __init__(self, pde, T_end, layer_dims, reference_timesteps, ref_algo, reference_params=None, learning_rate=0.001, activation='relu', **kwargs):
        super().__init__(**kwargs)

        self.modeltype = f"ANN_Model with layers {layer_dims} and {activation}-activation"

        self.T_end = float(T_end)
        self.pde = pde
        self.layer_dims = layer_dims
        self.learning_rate = learning_rate
        self.ref_algo = ref_algo
        self.ref_params = reference_params
        self.reference_timesteps = reference_timesteps

        self.nr_spacediscr = pde.nr_spacediscr
        self.spacedim = pde.space_dim
        self.coarse_spacediscr = pde.coarse_spacediscr
        self.coarse_spacedim = pde.coarse_space_dim
        self.refine_factor = self.nr_spacediscr // self.coarse_spacediscr

        self.project_matrix = project_matrix(self.coarse_spacediscr, self.refine_factor)

        # Create ANN
        self.model = keras.Sequential()
        self.model.add(keras.Input(shape=(layer_dims[0],), dtype=tf.float64))
        for l in layer_dims[1:-1]:
            self.model.add(keras.layers.Dense(l, activation=activation, dtype=tf.float64))
        self.model.add(keras.layers.Dense(layer_dims[-1], dtype=tf.float64))

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

        self.evaluations = []

        self.reference_error = tf.Variable(1., dtype=tf.float64, trainable=False)

        self.initializer = GRF_2d(self.nr_spacediscr)

    def call(self, initial_values):
        '''
            initial_values: [batch_size, space_dim]
        '''
        u = tf.cast(initial_values, tf.float64)
        u = tf.matmul(u, self.project_matrix)
        return self.model(u)


def loss(predictions, labels):
    '''
        prediction, labels: [batch_size, space_dim]
    '''
    labels_float = tf.cast(labels, tf.float64)
    loss_by_batch = tf.reduce_mean(tf.square(predictions - labels_float), axis=1)
    return tf.reduce_mean(loss_by_batch)


def difference_loss(prediction, base, labels, reference_error):
    loss_by_batch = tf.reduce_mean(tf.square(tf.sqrt(reference_error) * prediction + base - labels), axis=1)
    loss_by_batch /= reference_error
    return tf.reduce_mean(loss_by_batch)


def loss_by_data(prediction, labels):
    loss_by_batch = tf.reduce_mean(tf.square(prediction - labels), axis=1, keepdims=True)
    return loss_by_batch


def train_step(model, batch_size, from_data=False, ref_data=None):

    if from_data:
      initial_data, final_data = ref_data
      initial_value, reference_sol = get_batch(initial_data, final_data, batch_size)
    else:
      initial_value = model.initializer.sample(batch_size)
      reference_sol = generate_reference_solution(model, initial_value)

    reference_sol = tf.matmul(reference_sol, model.project_matrix)
    with tf.GradientTape() as tape:
        predictions = model.call(initial_value)
        loss_value = loss(predictions, reference_sol)

    grads = tape.gradient(loss_value, model.trainable_weights)
    model.optimizer.apply_gradients(zip(grads, model.trainable_weights))

    return tf.sqrt(loss_value)


def multiphase_training(model, max_steps, evaluate_steps, batch_size, initial_learning_rate, tolerance=10,
                        verbose=False, file=None, from_data=False, ref_data=None):
    last_time_improved = 0
    best_loss = 1000.
    current_learn_rate = initial_learning_rate
    model.optimizer.learning_rate.assign(initial_learning_rate)
    for j in range(max_steps):
        new_error = train_step(model, batch_size, from_data, ref_data)
        model.evaluations.append(new_error)
        if ((j+1) % evaluate_steps == 0) and verbose:
            print(f'{j + 1} train steps completed. Current error: {new_error}.', file=file)
            print(f'{j + 1} train steps completed. Current error: {new_error}.')
        if new_error < best_loss:
            best_loss = new_error
            last_time_improved = 0
        else:
            if last_time_improved == tolerance:
                current_learn_rate *= 0.8
                model.optimizer.learning_rate.assign(current_learn_rate)
                if verbose:
                    print(f'{j+1} train steps completed. New learning rate: {current_learn_rate}', file=file)
                    print(f'{j+1} train steps completed. New learning rate: {current_learn_rate}')
                last_time_improved = 0
            else:
                last_time_improved += 1

        if np.isnan(new_error.numpy()):
            break


def difference_train_step(model, batch_size, trained_base, from_data=False, ref_data=None):

    if from_data:
      initial_data, final_data = ref_data
      initial_value, reference_sol = get_batch(initial_data, final_data, batch_size)
    else:
      initial_value = model.initializer.sample(batch_size)
      reference_sol = generate_reference_solution(model, initial_value)
    base_sol = trained_base.call(initial_value)

    reference_sol = tf.matmul(reference_sol, trained_base.project_matrix)

    error_reference = loss(base_sol, reference_sol)
    model.reference_error.assign(tf.sqrt(error_reference))
    with tf.GradientTape() as tape:
        predictions = model.call(initial_value)
        loss_value = difference_loss(predictions, base_sol, reference_sol, error_reference)

    grads = tape.gradient(loss_value, model.trainable_weights)
    model.optimizer.apply_gradients(zip(grads, model.trainable_weights))

    return tf.sqrt(loss_value)


def multiphase_difference_training(model, trained_base, max_steps, evaluate_steps, batch_size,
                                    initial_learning_rate, tolerance=8, verbose=False, file=None,
                                   from_data=False, ref_data=None):
    last_time_improved = 0
    best_loss = 1000.
    current_learn_rate = initial_learning_rate
    model.optimizer.learning_rate.assign(initial_learning_rate)
    for j in range(max_steps):
        new_error = difference_train_step(model, batch_size, trained_base, from_data, ref_data)
        model.evaluations.append(new_error)
        if ((j+1) % evaluate_steps == 0) and verbose:
            print(f'{j + 1} difference train steps completed. Current error: {new_error}.', file=file)
            print(f'{j + 1} difference train steps completed. Current error: {new_error}.')
        if new_error < best_loss:
            best_loss = new_error
            last_time_improved = 0
        else:
            if last_time_improved == tolerance:
                current_learn_rate *= 0.8
                model.optimizer.learning_rate.assign(current_learn_rate)
                if verbose:
                    print(f'{j+1} difference train steps completed. New learning rate: {current_learn_rate}', file=file)
                    print(f'{j+1} difference train steps completed. New learning rate: {current_learn_rate}')
                last_time_improved = 0
            else:
                last_time_improved += 1

        if np.isnan(new_error.numpy()):
            break


def evaluate(model, batch_size, from_data=False, ref_data=None, with_l1=False):
    if from_data:
      initial_data, final_data = ref_data
      initial_value, reference_sol = get_batch(initial_data, final_data, batch_size)
    else:
      initial_value = model.initializer.sample(batch_size)
      reference_sol = generate_reference_solution(model, initial_value)

    reference_sol = tf.matmul(reference_sol, model.project_matrix)
    base_output = model.call(initial_value)
    base_error_l2 = tf.reduce_mean(tf.square(base_output - reference_sol))
    base_error_l2 = tf.sqrt(base_error_l2)
    
    if with_l1:
        help_error = tf.reduce_mean(tf.square(base_output - reference_sol), axis=1)
        base_error_l1 = tf.reduce_mean(tf.sqrt(help_error))
        
        return base_error_l1, base_error_l2
        
    else:

        return base_error_l2


def diff_evaluate(base_model, diff_model, batch_size, from_data=False, ref_data=None, with_l1=False):
    if from_data:
      initial_data, final_data = ref_data
      initial_value, reference_sol = get_batch(initial_data, final_data, batch_size)
    else:
      initial_value = base_model.initializer.sample(batch_size)
      reference_sol = generate_reference_solution(base_model, initial_value)

    base_output = base_model.call(initial_value)
    reference_sol = tf.matmul(reference_sol, base_model.project_matrix)
    base_error_l2 = tf.reduce_mean(tf.square(base_output - reference_sol))
    base_error_l2 = tf.sqrt(base_error_l2)
    
    whole_output = base_output + diff_model.reference_error * diff_model.call(initial_value)
    whole_error_l2 = tf.reduce_mean(tf.square(whole_output - reference_sol))
    whole_error_l2 = tf.sqrt(whole_error_l2)
    
    if with_l1:
        help_error = tf.reduce_mean(tf.square(base_output - reference_sol), axis=1)
        base_error_l1 = tf.reduce_mean(tf.sqrt(help_error))
        
        help_whole_error = tf.reduce_mean(tf.square(whole_output - reference_sol), axis=1)
        whole_error_l1 = tf.reduce_mean(tf.sqrt(help_whole_error))
        
        return base_error_l1, base_error_l2, whole_error_l1, whole_error_l2
        
    else:
        return base_error_l2, whole_error_l2


def test_rk2_diff_init_params(pde, T_end, net_timesteps, fno_dims, batch_size, test_size, init_rate, fno_rate, max_steps, max_diff_steps, alphas, betas, ref_timesteps,
                         ref_algo, ref_params, file_string, array_file, from_data=False, ref_data=None):

    initial_errors = np.zeros((len(alphas), len(betas)))
    adann_errors = np.zeros((len(alphas), len(betas)))
    relative_errors = np.zeros((len(alphas), len(betas)))
    fno_errors = np.zeros((len(alphas), len(betas)))
    relative_fno_errors = np.zeros((len(alphas), len(betas)))

    with open(file_string, 'w+') as ff:
        print(f'Solving {pde.ODE_name} with implicit 2nd order Runge-Kutta method.', file=ff)
        print(f'RK2_Model with depth {net_timesteps} trained on coarser {pde.coarse_spacediscr}x{pde.coarse_spacediscr}-grid', file=ff)
        print(f'Difference FNO has depth {fno_dims[2]}, width {fno_dims[1]} and {fno_dims[0]} Fourier nodes.', file=ff)
        print(f'Final time {T_end}, batch size {batch_size}.', file=ff)
        print(f'Reference algo: {ref_algo} with {ref_timesteps} steps.', file=ff)
        print(f'Use adaptive learning rate, initial lr={init_rate} and {max_steps} ADAM steps for base, lr={fno_rate} and {max_diff_steps} ADAM steps for diff training.', file=ff)
        print(f'Compute test errors on independent dataset of size {test_size}.', file=ff)
        print(f'Format: initial params, initial error (L1, L2), error after base training (L1, L2),'
              f'error after FNO diff training (L1, L2), '
              f'relative improvement during base training, rel. improv. during ANN diff training, overall improvement compared to [0.5, 0.5] base RK2 model.', file=ff)
        
        adann = PDE_RK2_model(pde, T_end, net_timesteps, ref_timesteps, [0.5, 0.5], ref_algo, ref_params, train_refine=False)
        print('Starting evaluation of reference model.')
        ref_error = evaluate(adann, test_size, from_data=from_data, ref_data=ref_data)
        print(f'Reference error is {ref_error}', file=ff)
        print(f'Reference error is {ref_error:.6f}')

        start_time = time.perf_counter()
        for i in range(len(alphas)):
            for j in range(len(betas)):
                params = [alphas[i], betas[j]]
                print(f'Initializing base model with params {params}.')
                adann = PDE_RK2_model(pde, T_end, net_timesteps, ref_timesteps, params, ref_algo, ref_params, train_refine=False)
                fno = FNO_Model(pde, T_end, fno_dims, ref_timesteps, ref_algo)                

                initial_time = time.perf_counter()
                initial_error_l1, initial_error_l2 = evaluate(adann, test_size, from_data=from_data, ref_data=ref_data, with_l1=True)
                initial_errors[i, j] = initial_error_l2

                first_init_time = time.perf_counter()
                print(f'Time to evaluate initial base model: {first_init_time-initial_time:.4f} seconds.')
                multiphase_training(adann, max_steps, 10, batch_size, init_rate, from_data=from_data, ref_data=ref_data)
                first_train_time = time.perf_counter()
                print(f'Time to train base model: {first_train_time - first_init_time:.4f} seconds.')

                adann_error_l1, adann_error_l2 = evaluate(adann, test_size, from_data=from_data, ref_data=ref_data, with_l1=True)
                relative_error = adann_error_l2 / initial_error_l2
                adann_errors[i, j] = adann_error_l2
                relative_errors[i, j] = relative_error
                second_init_time = time.perf_counter()
                print(f'Time to evaluate trained base model: {second_init_time - first_train_time:.4f} seconds.')

                multiphase_difference_training(fno, adann, max_diff_steps, 10, batch_size, fno_init_rate, from_data=from_data, ref_data=ref_data)
                second_train_time = time.perf_counter()
                print(f'Time to train difference model: {second_train_time - second_init_time:.4f} seconds.')
                _, _, fno_error_l1, fno_error_l2 = diff_evaluate(adann, fno, test_size, from_data=from_data,
                                                                 ref_data=ref_data, with_l1=True)
                relative_fno_error = fno_error_l2 / adann_error_l2
                fno_errors[i, j] = fno_error_l2
                relative_fno_errors[i, j] = relative_fno_error
                third_eval_time = time.perf_counter()
                print(f'Time to evaluate full model: {third_eval_time - second_train_time:.4f} seconds.')

                print(f'{params}, {initial_error_l1:.6f}, {initial_error_l2:.6f}, {adann_error_l1:.6f}, {adann_error_l2:.6f}, '
                      f'{fno_error_l1:.6f}, {fno_error_l2:.6f},  {100 * relative_error:.2f}%, {100 * relative_fno_error:.2f}%, '
                      f'{100 * fno_error_l2 / ref_error:.2f}%')
                print(f'{params}, {initial_error_l1}, {initial_error_l2}, {adann_error_l1}, {adann_error_l2}, '
                      f'{fno_error_l1}, {fno_error_l2},  {100 * relative_error:.2f}%, {100 * relative_fno_error:.2f}%, '
                      f'{100 * fno_error_l2 / ref_error:.2f}%', file=ff)
                
        end_time = time.perf_counter()
        average_time = (end_time - start_time) / (len(alphas) * len(betas))
        print(f'Average time to train & test a single model: {average_time:.3f} seconds.', file=ff)
        print(f'Every model has {count_params(adann.trainable_weights)} base and {count_params(fno.trainable_weights)} diff trainable variables.', file=ff)

        initial_errors_masked = np.ma.masked_invalid(initial_errors)
        adann_errors_masked = np.ma.masked_invalid(adann_errors)
        relative_errors_masked = np.ma.masked_invalid(relative_errors)
        fno_errors_masked = np.ma.masked_invalid(fno_errors)
        relative_fno_errors_masked = np.ma.masked_invalid(relative_fno_errors)

        i_initial, j_initial = np.unravel_index(np.argmin(initial_errors_masked), (len(alphas), len(betas)))
        print(f'Minimal initial error {initial_errors_masked[i_initial, j_initial]} reached for params'
              f' alpha={alphas[i_initial]}, beta={betas[j_initial]}', file=ff)

        i_adann, j_adann = np.unravel_index(np.argmin(adann_errors_masked), (len(alphas), len(betas)))
        print(f'Minimal base error {adann_errors_masked[i_adann, j_adann]} reached for params'
              f' alpha={alphas[i_adann]}, beta={betas[j_adann]}', file=ff)

        i_fno, j_fno = np.unravel_index(np.argmin(fno_errors_masked), (len(alphas), len(betas)))
        print(f'Minimal fno error {fno_errors_masked[i_fno, j_fno]} reached for params'
              f' alpha={alphas[i_fno]}, beta={betas[j_fno]}', file=ff)

        i_rel_adann, j_rel_adann = np.unravel_index(np.argmin(relative_errors_masked), (len(alphas), len(betas)))
        print(f'Optimal base improvement {100 * relative_errors_masked[i_rel_adann, j_rel_adann]:.2f}% '
              f'reached for params alpha={alphas[i_rel_adann]}, beta={betas[j_rel_adann]}', file=ff)

        i_rel_fno, j_rel_fno = np.unravel_index(np.argmin(relative_fno_errors_masked), (len(alphas), len(betas)))
        print(f'Optimal FNO improvement {100 * relative_fno_errors_masked[i_rel_fno, j_rel_fno]:.2f}% '
              f'reached for params alpha={alphas[i_rel_fno]}, beta={betas[j_rel_fno]}', file=ff)

    np.savez(array_file, initial_errors=initial_errors, adann_errors=adann_errors, relative_errors=relative_errors,
             fno_errors=fno_errors, relative_fno_errors=relative_fno_errors)


In [ ]:
N = 80
coarse_N = 40
lap_factor = 0.01
T_end = 2.
nr_timesteps = 200
sqrt_pde = semilinear_heat_PDE_FDM(N, coarse_N, lap_factor, sqrt_nonlin, 'Sqrt-Lip')

In [7]:
net_timesteps = 2

reference_params = [0.5, 0.5]

fno_depth = 5
fno_dims = [20, 80, fno_depth]

ann_dims = [coarse_N ** 2, 2 * coarse_N ** 2, 2 * coarse_N ** 2, 2 * coarse_N ** 2, coarse_N ** 2]
ann_activation = 'gelu'

ref_algo = 'RK2'

In [18]:
adann_init_rate = 0.00002
fno_init_rate = 0.002
ann_init_rate = 0.002
batch_size = 128
test_size = 1024
max_steps = 1000
eval_steps = 10

In [ ]:
# generating the reference solution
sol_size = 16 * 1024
F = GRF_2d(N)
w0 = F.sample(sol_size)
print(f'Starting to solve.')
time0 = time.perf_counter()
w_ref = solve_2d_pde_rk2(sqrt_pde, w0, T_end, nr_timesteps, [0.5, 0.5], verbose=True)
time1 = time.perf_counter()
print(f'Total time to generate solution: {time1 - time0:.3f} seconds.')
print(f'Average time per solution step: {(time1 - time0) / nr_timesteps:.3f} seconds.')

In [11]:
sq_data = [w0, w_ref]  # training samples

In [26]:
fno = FNO_Model(sqrt_pde, T_end, fno_dims, nr_timesteps, ref_algo) 

In [19]:
ann = ANN_Model(sqrt_pde, T_end, ann_dims, nr_timesteps, ref_algo, activation=ann_activation)

In [ ]:
# Training the FNO as comparison
with open('TestFNO.txt', 'w+') as ff:
    start_time = time.perf_counter()
    multiphase_training(fno, max_steps, eval_steps, batch_size, fno_init_rate, verbose=True, file=ff, from_data=True, ref_data=sq_data)
    end_time = time.perf_counter()
    
    print(f'{end_time-start_time:.2f} seconds.', file=ff)
    
print(f'{end_time-start_time:.2f} seconds.')

In [ ]:
# Training the ANN as comparison
with open('TestANN.txt', 'w+') as ff:
    start_time = time.perf_counter()
    multiphase_training(ann, 10, eval_steps, batch_size, 0.001, verbose=True, file=ff, from_data=True, ref_data=sq_data)
    end_time = time.perf_counter()
    
    print(f'{end_time-start_time:.3f} seconds.', file=ff)
    
print(f'{end_time-start_time:.3f} seconds.')

In [ ]:
print(count_params(fno.trainable_weights))
eval_time1 = time.perf_counter()
a, b = evaluate(fno, test_size, from_data=True, ref_data=sq_data, with_l1=True)
eval_time2 = time.perf_counter()
print(a, b)
print(f'{eval_time2 - eval_time1:.4f} seconds.')

In [ ]:
print(count_params(ann.trainable_weights))
eval_time1 = time.perf_counter()
a, b = evaluate(ann, test_size, from_data=True, ref_data=sq_data, with_l1=True)
eval_time2 = time.perf_counter()
print(a, b)
print(f'{eval_time2 - eval_time1:.4f} seconds.')

In [ ]:
## training full ADANNs for different initial parameters alphas, betas
batch_size = 64
test_size = 512
fno_dims = [15, 30, 4]
max_steps = 200
max_diff_steps = 200

file_string = f'SQ_RK2_DiffFNO_TestInitParamsRefineTrain_{N}_{coarse_N}_{batch_size}_{net_timesteps}_{nr_timesteps}_b.txt'
array_file = f'SQ_RK2_DiffFNO_TestInitParamsRefineTrain_{N}_{coarse_N}_{batch_size}_{net_timesteps}_{nr_timesteps}_b.npz'


alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
betas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

test_rk2_diff_init_params(sqrt_pde, T_end, net_timesteps, fno_dims, batch_size, test_size, adann_init_rate, fno_init_rate,
                              max_steps, max_diff_steps, alphas, betas, nr_timesteps, ref_algo, reference_params,
                              file_string, array_file, True, sq_data)

In [ ]:
## plotting the errors as heat maps

import matplotlib.pyplot as plt

data = np.load('SQ_RK2_DiffFNO_TestInitParamsRefineTrain_80_40_64_2_200.npz')

initial_errors = data['initial_errs']
base_errors = data['base_errs']
full_errors = data['full_errs']

grid_p1_low = 0.1
grid_p1_high = 0.9
grid_nr_p1_steps = 9

grid_p2_low = 0.2
grid_p2_high = 0.9
grid_nr_p2_steps = 8

p1_values = np.linspace(grid_p1_low, grid_p1_high, grid_nr_p1_steps)
p2_values = np.linspace(grid_p2_low, grid_p2_high, grid_nr_p2_steps)

alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
betas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

grid_p1s, grid_p2s = np.meshgrid(alphas, betas)
start_errors_masked = np.ma.masked_invalid(np.transpose(initial_errors))
err_min, err_max = start_errors_masked[1:].min(), start_errors_masked[1:].max()


fig, ax = plt.subplots()
c = ax.pcolormesh(grid_p1s[1:], grid_p2s[1:], start_errors_masked[1:], cmap='RdBu', vmin=err_min, vmax=err_max)
ax.set_title('Start errors')
ax.set_xlabel(r'$p_1$')
ax.set_ylabel(r'$p_2$')
fig.colorbar(c, ax=ax)
fig.savefig('Grid_Initial_Errors_v3.pdf')
plt.show()



end_errors_base_masked = np.ma.masked_invalid(np.transpose(base_errors))
err_min, err_max = end_errors_base_masked[1:].min(), end_errors_base_masked[1:].max()
fig, ax = plt.subplots()
c = ax.pcolormesh(grid_p1s[1:], grid_p2s[1:], end_errors_base_masked[1:], cmap='RdBu', vmin=err_min, vmax=err_max)
ax.set_title('Trained base errors')
ax.set_xlabel(r'$p_1$')
ax.set_ylabel(r'$p_2$')
fig.colorbar(c, ax=ax)
fig.savefig('Grid_Base_Errors_v3.pdf')
plt.show()



end_errors_masked = np.ma.masked_invalid(np.transpose(full_errors))
err_min, err_max = end_errors_masked[1:].min(), end_errors_masked[1:].max()
fig, ax = plt.subplots()
c = ax.pcolormesh(grid_p1s[1:], grid_p2s[1:], end_errors_masked[1:], cmap='RdBu', vmin=err_min, vmax=err_max)
ax.set_title('Trained full ADANN errors')
ax.set_xlabel(r'$p_1$')
ax.set_ylabel(r'$p_2$')
fig.colorbar(c, ax=ax)
fig.savefig('Grid_Full_Errors_v3.pdf')
plt.show()